In [ ]:
# %%writefile word2vec.py
from util import *
from gensim.models import Word2Vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

w2v_size=50

class Playlists:
    def __iter__(self):
        i_path='raw/mpd/data'
        slices=listdir(i_path)
        for i in range(len(slices)):
            s=slices[i]
            with open(join(i_path,s)) as f:
                plist=json.load(f)
                for p in plist['playlists']:
                    line = p['name']
                    for t in p['tracks']:
                        line += ' ' + t['track_name']

                    yield [w for w in build_title(line).lower().split() if w.isalpha()]
                

model=Word2Vec(Playlists(),size=w2v_size)
model.save('w2v/word2vec_gensim')
model.wv.save_word2vec_format('w2v/word2vec_org','w2v/vocabulary',binary=False)

In [ ]:
len(model.wv.vocab)

In [ ]:

tracks=pd.read_csv('raw/large/tracks.csv')
tracks_vec=np.zeros((tracks.shape[0],w2v_size))

zero_tracks=0
track_names=tracks.track_name
track_ids=tracks.track_id
for i in tnrange(tracks.shape[0]):
    title=build_title(track_names[i])
    words=[w for w in title.lower().split() if w.isalpha()]
    z=0
    for w in words:
        if w in model.wv:
            tracks_vec[track_ids[i]]+=model.wv[w]
            z=1
    if z==0:
        zero_tracks += 1

print(zero_tracks/tracks.shape[0])

In [ ]:
model.wv.most_similar('countryyy')

In [ ]:
model.wv.most_similar('home')

In [ ]:
playlists_tra=pd.read_csv('raw/large/playlists.csv')
playlists_tst=pd.read_csv('raw/test/playlists.csv')

In [ ]:

playlists=pd.concat([playlists_tra,playlists_tst])
playlist_vecs=np.zeros((playlists.shape[0],model.wv['home'].shape[0]))
plists=playlists.sort_values('pid')
for i in tnrange(plists.shape[0]):
    words=[w for w in build_title(plists.iloc[i]['name']).lower().split() if w.isalpha() and w != 'nan']
    c=0
    for w in words:
        if w in model.wv:
            playlist_vecs[i,:]+=model.wv[w]
    if c > 0:
        playlist_vecs[i,:]/=c
        


In [ ]:
np.save('raw/w2v/playlist_vecs',playlist_vecs)

In [ ]:
track_vecs=np.zeros((tracks.shape[0],model.wv['home'].shape[0]))
plists=tracks.sort_values('track_id')
for i in tnrange(plists.shape[0]):
    words=[w for w in build_title(plists.iloc[i]['track_name']).lower().split() if w.isalpha()]
    c=0
    for w in words:
        if w in model.wv:
            track_vecs[i,:]+=model.wv[w]
    if c > 0:
        track_vecs[i,:]/=c
        


In [ ]:
np.save('raw/w2v/track_vecs',track_vecs)